## 기존 충전소 관련 입지 스코어링 모델

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from haversine import haversine
from sklearn.preprocessing import MinMaxScaler

In [6]:
temp1 = pd.read_csv('일평균충전량_충전소.csv')
temp1.iloc[:,[1,2,4,6]]

,충전소명,충전량,마지막_측정일,일일 평균 충전량
0,DMC래미안e편한세상,77453.31,2022-03-31,169.853750
1,DMC센트럴아이파크아파트,73202.01,2022-03-31,160.530724
2,DMC아이파크 아파트,5679.27,2022-03-28,12.733789
3,GS강동자이 아파트,28311.00,2022-03-31,62.221978
4,LG한강자이 아파트,26309.74,2022-03-31,57.696798
...,...,...,...,...
776,휘경주공1단지,77917.32,2022-03-31,170.871316
777,흑석한강센트레빌,27620.62,2022-03-30,60.838370
778,힐스테이트 강동리버뷰,27111.04,2022-03-31,59.584703
779,힐스테이트상도프레스티지,19231.28,2022-03-31,42.266549


In [7]:
sample_data = pd.read_csv('sample.csv')
new_yx = sample_data.iloc[:,1:3].copy()
df = pd.read_excel('charger.xlsx')
df_charger = df.iloc[:, [2,3]].copy()
df_charger['avg_charger'] = temp1['일일 평균 충전량']

In [8]:
# 기존 충전소 위경도 + 일평균 충전량
df_charger

,위도,경도,avg_charger
0,37.572233,126.909946,169.853750
1,37.579362,126.920840,160.530724
2,37.575441,126.914327,12.733789
3,37.537527,127.148702,62.221978
4,37.519278,126.970777,57.696798
...,...,...,...
776,37.582369,127.071263,170.871316
777,37.505913,126.965193,60.838370
778,37.548820,127.125370,59.584703
779,37.494751,126.948272,42.266549


In [9]:
# 입지 후보지 위경도
new_yx

,위도,경도
0,37.479132,127.097849
1,37.646592,127.046026
2,37.521695,127.067788
3,37.545310,127.186173
4,37.637137,127.068117
5,37.584924,126.967558
6,37.615603,126.997116
7,37.628316,127.081170
8,37.584854,127.000622
9,37.602030,126.970806


In [10]:
class charger_score:
    # dat = 기존 충전소의 (y, x, 충전량)을 column으로 가지고 있는 dataframe.
    # new_dat = (new_y, new_x)를 column으로 가지고 있는 dataframe.
    def __init__(self, dat, new_dat):
        self.dat = dat
        self.new_dat = new_dat
        from haversine import haversine
        import pandas as pd
        import numpy as np
    # 기준 거리 이내의 충전소 개수/충전량/가장 가까운 충전소까지의 거리를 계산하는 함수
    def charger_dist(self, distance = 1):
        result = pd.DataFrame()
        for i in range(self.new_dat.shape[0]):
            cnt = 0
            charge = 0
            shortest = 1000
            for j in range(self.dat.shape[0]):
                old = (self.dat.iloc[j,0], self.dat.iloc[j,1])
                new = (self.new_dat.iloc[i,0], self.new_dat.iloc[i,1])
                dist = haversine(old, new, unit='km')
                shortest = np.minimum(shortest, dist)
                if dist < distance:
                    cnt += 1
                    charge += self.dat.iloc[i,2]
            # 포인트 별 결과 = (new_y, new_x, (기준 이내 거리의)개수, 평균 충전량, 가장 가까운 충전소까지의 거리)
            if cnt == 0:
                avg = 0
            else:
                avg = charge/cnt
            temp = pd.DataFrame([[self.new_dat.iloc[i,0], self.new_dat.iloc[i,1], cnt, avg, shortest]], columns=['y','x','cnt','charge', 'shortest'])
            result = pd.concat([result,temp], ignore_index=True)
        return result
    
    # score 계산용 함수 (positive관계)
    def get_pscore4(self, x, q1, q2, q3):
        if x <= q1:
            score = 1
        elif x <= q2:
            score = 2
        elif x <= q3:
            score = 3
        else:
            score = 4
        return score
    # score 계산용 함수 (negative관계)
    def get_nscore4(self, x, q1, q2, q3):
        if x <= q1:
            score = 4
        elif x <= q2:
            score = 3
        elif x <= q3:
            score = 2
        else:
            score = 1
        return score
    # x : scoring 대상 dataframe
    # crit : 기준
    # score 계산용 함수 (positive관계)
    def get_pscore10(self, x, crit):
        bins = list(crit)
        bins.append(1e10)
        bins.insert(0, -1)
        bins_label = np.arange(1,11,1)
        temp = pd.cut(x, bins, right=False, labels=bins_label)
        return temp.astype('int') # dataframe
    # score 계산용 함수 (negative관계)
    def get_nscore10(self, x, crit):
        bins = list(crit)
        bins.append(1e10)
        bins.insert(0, -1)
        bins_label = np.arange(10,0,-1)
        temp = pd.cut(x, bins, right=False, labels=bins_label)
        return temp.astype('int') # dataframe
    
    # 10분위수를 계산하기 위한 함수(양 끝 edge 포함)
    def criteria(self):
        df = self.dat.iloc[:,2].copy()
        temp = df.quantile(np.arange(0.1,1,0.1))
        return temp
    # 최종 score 적용
    def score(self, distance = 1, cnt_criteria = [1,3,5], dist_criteria = [0.25, 0.5, 0.75],  w = [0.4, 0.2, 0.4]):
        res = self.charger_dist(distance = distance)
        res['cnt_score'] = res.iloc[:,2].apply(lambda x: self.get_nscore4(x, cnt_criteria[0], cnt_criteria[1], cnt_criteria[2]))
        res['charge_score'] = self.get_pscore10(res.iloc[:,3], self.criteria())
        res['dist_score'] = res.iloc[:,4].apply(lambda x: self.get_pscore4(x, dist_criteria[0], dist_criteria[1], dist_criteria[2]))
        res['score'] = res['cnt_score']*(10/4)*w[0] +  res['charge_score']*(10/10)*w[1] + res['dist_score']*(10/4)*w[2]
        return res

In [11]:
model_charger = charger_score(df_charger, new_yx)
# 범위 내 충전소 개수/충전량/가장 가까운 충전소 거리 결과 도출
model_charger.charger_dist(distance = 1)

,y,x,cnt,charge,shortest
0,37.479132,127.097849,3,169.853750,0.567452
1,37.646592,127.046026,6,160.530724,0.244193
2,37.521695,127.067788,6,12.733789,0.607808
3,37.545310,127.186173,0,0.000000,1.356196
4,37.637137,127.068117,8,57.696798,0.329988
5,37.584924,126.967558,1,197.872917,0.891021
6,37.615603,126.997116,3,249.866593,0.318225
7,37.628316,127.081170,12,55.793385,0.097310
8,37.584854,127.000622,3,117.677478,0.127432
9,37.602030,126.970806,0,0.000000,1.216268


In [12]:
# 기존 충전소 관련 입지 스코어 결과
model_charger.score(distance=1).sort_values(by = 'score')

,y,x,cnt,charge,shortest,cnt_score,charge_score,dist_score,score
16,37.497316,127.018676,9,24.540000,0.075815,1,5,1,3.0
7,37.628316,127.081170,12,55.793385,0.097310,1,7,1,3.4
1,37.646592,127.046026,6,160.530724,0.244193,1,10,1,4.0
20,37.606333,127.101728,6,0.037830,0.519686,1,1,3,4.2
4,37.637137,127.068117,8,57.696798,0.329988,1,7,2,4.4
11,37.475951,127.038580,5,2.392200,0.469148,2,2,2,4.4
2,37.521695,127.067788,6,12.733789,0.607808,1,3,3,4.6
10,37.521659,127.123254,6,15.152500,0.745259,1,4,3,4.8
14,37.497601,127.026742,6,39.979778,0.649799,1,6,3,5.2
15,37.498785,126.872557,5,79.556601,0.264552,2,8,2,5.6


In [13]:
model_charger.score(distance=1).iloc[[16,7,13,5],:]

,y,x,cnt,charge,shortest,cnt_score,charge_score,dist_score,score
16,37.497316,127.018676,9,24.540000,0.075815,1,5,1,3.0
7,37.628316,127.081170,12,55.793385,0.097310,1,7,1,3.4
13,37.512547,126.988023,2,113.209253,0.811247,3,9,4,8.8
5,37.584924,126.967558,1,197.872917,0.891021,4,10,4,10.0


In [14]:
charger_dat = model_charger.score(distance = 1)

In [123]:
# feature score과 charger score를 합쳐주는 클래스
class final_score:
    # feature_dat = feature variable을 통한 scaled_scoring 결과로, (y, x, feature_score)를 column으로 가지고 있는 dataframe.
    # charger_dat = (y, x, charger_score)를 column으로 가지고 있는 dataframe.
    def __init__(self, feature_dat, charger_dat):
        self.feature_dat = feature_dat
        self.charger_dat = charger_dat
        import pandas as pd
        import numpy as np
        from sklearn.preprocessing import MinMaxScaler
    def score(self, w = [0.5, 0.5]):
        result = self.feature_dat.copy()
        result['charger_score'] = self.charger_dat.iloc[:,2]
        result['final_score'] = result['feature_score']*w[0] + result['charger_score']*w[1]
        return result

In [94]:
feature_dat = pd.read_excel('predict_dataframe.xlsx')
feature_dat = feature_dat.iloc[:,[1,2,13]]
feature_dat

,위도,경도,pred
0,37.479132,127.097849,0.418935
1,37.646592,127.046026,1.383698
2,37.521695,127.067788,0.444848
3,37.545310,127.186173,0.341908
4,37.637137,127.068117,1.505358
5,37.584924,126.967558,0.458843
6,37.615603,126.997116,0.446516
7,37.628316,127.081170,1.338870
8,37.584854,127.000622,1.255518
9,37.602030,126.970806,0.416345


In [83]:
charger_dat = model_charger.score(distance = 1)
charger_dat = charger_dat.iloc[:,[0,1,8]]
charger_dat

,y,x,score
0,37.479132,127.097849,8.0
1,37.646592,127.046026,4.0
2,37.521695,127.067788,4.6
3,37.545310,127.186173,8.2
4,37.637137,127.068117,4.4
5,37.584924,126.967558,10.0
6,37.615603,126.997116,7.0
7,37.628316,127.081170,3.4
8,37.584854,127.000622,5.8
9,37.602030,126.970806,8.2


In [100]:
final_model = final_score(feature_dat, charger_dat)

In [101]:
final_model.feature_scaler()

,feature_score
0,1.600198
1,8.818104
2,1.794070
3,1.023922
4,9.728306
5,1.898770
6,1.806551
7,8.482720
8,7.859123
9,1.580825


In [103]:
final_model.score().sort_values(by = 'final_score')

,위도,경도,feature_score,charger_score,final_score
2,37.521695,127.067788,1.794070,4.6,3.197035
6,37.615603,126.997116,1.806551,7.0,4.403275
23,37.525893,127.098955,1.000000,8.2,4.600000
3,37.545310,127.186173,1.023922,8.2,4.611961
26,37.433896,126.887180,1.078788,8.2,4.639394
21,37.437236,127.061465,1.109998,8.2,4.654999
25,37.635217,126.937753,1.278533,8.2,4.739266
0,37.479132,127.097849,1.600198,8.0,4.800099
9,37.602030,126.970806,1.580825,8.2,4.890413
17,37.558914,127.104205,1.846239,8.2,5.023120
